In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join("../.."))
sys.path.append(module_path)

import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

from pvi.models.bnn import TwoLayerClassificationBNN
from pvi.clients.synchronous_client import SynchronousClient
from pvi.servers.sequential_server import SequentialServer

from pvi.distributions.exponential_family_distributions import MeanFieldGaussianDistribution
from pvi.distributions.exponential_family_factors import MeanFieldGaussianFactor

NameError: name 'abstractproperty' is not defined

In [2]:
# ! python ../../data/preprocess_data.py --dir=./

0/4177 rows had missing data

0/32561 rows had missing data

0/16281 rows had missing data

2480/8124 rows had missing data

37/690 rows had missing data

0/45211 rows had missing data

0/21263 rows had missing data

0/45730 rows had missing data

0/9567 rows had missing data

Processing abalone dataset
Input  shape: (4177, 10)
Output shape: (4177, 1)
Processing adult dataset
Input  shape: (48842, 108)
Output shape: (48842, 1)
Processing mushroom dataset
Input  shape: (8124, 111)
Output shape: (8124, 1)
Processing credit dataset
Input  shape: (653, 46)
Output shape: (653, 1)
Processing bank dataset
Input  shape: (45211, 51)
Output shape: (45211, 1)
Processing superconductor dataset
Input  shape: (21263, 81)
Output shape: (21263, 1)
Processing protein dataset
Input  shape: (45730, 9)
Output shape: (45730, 1)
Processing power dataset
Input  shape: (9567, 4)
Output shape: (9567, 1)
